<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks<a class="jp-InternalAnchorLink" href="#unSkript-Runbooks" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success"><strong>To ensure the Redshift cluster has pause resume enabled in AWS using unSkript actions.</strong></div>
</center><center>
<h2 id="Ensure-Redshift-Clusters-have-Paused-Resume-Enabled">Ensure Redshift Clusters have Paused Resume Enabled<a class="jp-InternalAnchorLink" href="#Ensure-Redshift-Clusters-have-Paused-Resume-Enabled" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self">&para;</a></h1>
<ol>
<li>AWS Find Redshift Cluster without Pause Resume Enabled</li>
<li>AWS Schedule Redshift Cluster Pause Resume Enabled</li>
</ol>

In [46]:
if redshift_clusters and not region:
    raise SystemExit("Provide region for redshift_clusters!")
if region == None:
    region = ""

<h3 id="AWS-Find-Redshift-Cluster-without-Pause-Resume-Enabled">AWS Find Redshift Cluster without Pause Resume Enabled<a class="jp-InternalAnchorLink" href="#AWS-Find-Redshift-Cluster-without-Pause-Resume-Enabled" target="_self">&para;</a></h3>
<p>Here we will use unSkript <strong>AWS Find Redshift Cluster without Pause Resume Enabled</strong> action. This action filters all the redshift clusters from the given region and returns a list of clusters that don't have pause resume enabled.</p>
<blockquote>
<p>Input parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable:&nbsp;<code>clusters</code></p>
</blockquote>

In [ ]:
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Tuple
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype
@beartype
def aws_find_redshift_cluster_without_pause_resume_enabled_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_find_redshift_cluster_without_pause_resume_enabled(handle, region: str = "") -> Tuple:
    """aws_find_redshift_cluster_without_pause_resume_enabled Gets all redshift cluster which don't have pause and resume not enabled.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: AWS Region.

        :rtype: Tuple with the status result and a list of all redshift clusters that don't have pause and resume enabled.
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            redshift_Client = handle.client('redshift', region_name=reg)
            response = aws_get_paginator(redshift_Client, "describe_clusters", "Clusters")
            for cluster in response:
                cluster_dict = {}
                cluster_name = cluster["ClusterIdentifier"]
                schedule_actions = aws_get_paginator(redshift_Client, "describe_scheduled_actions", "ScheduledActions",Filters=[{'Name': 'cluster-identifier', 'Values': [cluster_name]}])

                if schedule_actions:
                    for actions in schedule_actions:
                        if "ResumeCluster" in actions["TargetAction"].keys() or "PauseCluster" in actions["TargetAction"].keys():
                            pass
                        else:
                            cluster_dict["cluster_name"] = cluster_name
                            cluster_dict["region"] = reg
                            result.append(cluster_dict)
                else:
                    cluster_dict["cluster_name"] = cluster_name
                    cluster_dict["region"] = reg
                    result.append(cluster_dict)
        except Exception as error:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not redshift_clusters",
    "condition_result": true
    }''')

task.configure(outputName="clusters")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_find_redshift_cluster_without_pause_resume_enabled, lego_printer=aws_find_redshift_cluster_without_pause_resume_enabled_printer, hdl=hdl, args=args)

<h3 id="Get-IAM-Role">Get IAM Role<a class="jp-InternalAnchorLink" href="Ensure_Redshift_Clusters_have_Paused_Resume_Enabled.ipynb#Get-IAM-Role" target="_self">&para;</a></h3>
<p>In this action, we use&nbsp;<strong>Run Command via AWS CLI</strong>&nbsp;action to get IAM role ARN for schedule pause resume.</p>
<blockquote>
<p>Output variable:&nbsp;<code>iam_role_arn</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_execute_cli_command_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_execute_cli_command(handle, aws_command: str) -> str:

    result = handle.aws_cli_command(aws_command)
    if result is None or result.returncode != 0:
        print(
            f"Error while executing command ({aws_command}): {result}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "aws_command": "\\"aws iam get-role --role-name scheduler.redshift.amazonaws.com --query 'Role.Arn' --output text\\""
    }''')
task.configure(outputName="iam_role_arn")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_execute_cli_command, lego_printer=aws_execute_cli_command_printer, hdl=hdl, args=args)

<h3 id="Modify-Step-1-Output&para;">Modify Step-1 Output</h3>
<p>In this action, we modify the output from step 1 and return a list of dictionaries for schedule pause resume in the redshift cluster.</p>
<blockquote>
<p>Output variable:&nbsp;<code>schedule_cluster_details</code></p>
</blockquote>

In [54]:
schedule_cluster_details = []
try:
    if clusters[0] == False:
        for instance in clusters[1]:
            instance["iam_role"] = iam_role_arn
            instance["pause_schedule_expression"] = pause_schedule_expression
            instance["resume_schedule_expression"] = resume_schedule_expression
            schedule_cluster_details.append(instance)
except Exception as e:
    for i in redshift_clusters:
        instance = {}
        instance["cluster_name"] = i
        instance["region"] = region
        instance["iam_role"] = iam_role_arn
        instance["pause_schedule_expression"] = pause_schedule_expression
        instance["resume_schedule_expression"] = resume_schedule_expression
        schedule_cluster_details.append(instance)
    else:
        raise Exception(e)

<p><strong>AWS Schedule Redshift Cluster Pause Resume Enabled</strong></p>
<p>In this action, we pass all details collected from the step1 and schedule pause resume redshift cluster.</p>
<blockquote>
<p>Input parameters: <code>iam_role_arn, cluster_name, region, pause_schedule_expression, resume_schedule_expression</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>schedule_info</code></p>
</blockquote>

In [ ]:
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, List
import pprint


from beartype import beartype
@beartype
def aws_schedule_pause_resume_enabled_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_schedule_pause_resume_enabled(handle,
                                      iam_role_arn: str,
                                      cluster_name: str,
                                      region: str,
                                      pause_schedule_expression: str,
                                      resume_schedule_expression: str) -> List:
    """aws_schedule_pause_resume_enabled Gets all Redshift clusters that don't have pause and resume enabled.

    :type handle: object
    :param handle: Object returned from task.validate(...).

    :type iam_role_arn: str
    :param iam_role_arn: The ARN of the IAM role.

    :type cluster_name: str
    :param cluster_name: The name of the Redshift cluster.

    :type region: str
    :param region: AWS Region.

    :type pause_schedule_expression: str
    :param pause_schedule_expression: The cron expression for the pause schedule.

    :type resume_schedule_expression: str
    :param resume_schedule_expression: The cron expression for the resume schedule.

    :rtype: List
    :return: A list of all Redshift clusters that don't have pause and resume enabled.
    """
    result = []
    pause_action_name = f"{cluster_name}-scheduled-pause"
    resume_action_name = f"{cluster_name}-scheduled-resume"

    try:
        redshift_client = handle.client('redshift', region_name=region)
        # Schedule pause action
        response_pause = redshift_client.create_scheduled_action(
            ScheduledActionName=pause_action_name,
            TargetAction={
                'PauseCluster': {'ClusterIdentifier': cluster_name}
            },
            Schedule=pause_schedule_expression,
            IamRole=iam_role_arn,
            Enable=True
        )
        print("Scheduled pause action:", response_pause)
        # Schedule resume action
        response_resume = redshift_client.create_scheduled_action(
            ScheduledActionName=resume_action_name,
            TargetAction={
                'ResumeCluster': {'ClusterIdentifier': cluster_name}
            },
            Schedule=resume_schedule_expression,
            IamRole=iam_role_arn,
            Enable=True
        )
        print("Scheduled resume action:", response_resume)

    except Exception as error:
        print(error)

    return result

task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter.get(\\"region\\")",
    "iam_role_arn": "iter.get(\\"iam_role\\")",
    "cluster_name": "iter.get(\\"cluster_name\\")",
    "pause_schedule_expression": "iter.get(\\"pause_schedule_expression\\")",
    "resume_schedule_expression": "iter.get(\\"resume_schedule_expression\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "schedule_cluster_details",
    "iter_parameter": ["region","iam_role_arn","cluster_name","pause_schedule_expression","resume_schedule_expression"]
    }''')
task.configure(outputName="schedule_info")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_schedule_pause_resume_enabled, lego_printer=aws_schedule_pause_resume_enabled_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we demonstrated using unSkript's AWS actions to enable Redshift clusters that don't have pause resume enabled and enable the pause resume to those clusters. To view the full platform capabunscriptedof unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>